<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Neural Networks

# Only do this once per VM, otherwise you'll get multiple clones and nested directories
!git clone https://github.com/jamestheengineer/data-science-from-scratch-Python.git
%cd data-science-from-scratch-Python/
!pip install import-ipynb
import import_ipynb

Cloning into 'data-science-from-scratch-Python'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 393 (delta 2), reused 0 (delta 0), pack-reused 387
Receiving objects: 100% (393/393), 1.06 MiB | 1.18 MiB/s, done.
Resolving deltas: 100% (242/242), done.
/content/data-science-from-scratch-Python
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=56fff1a8cf61df871bb51714f89666418c2cad74eadc17b2bafced0f7c249eba
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [2]:
from Chapter_04 import Vector, dot

def step_function(x: float) -> float:
  return 1.0 if x >= 0 else 0.0

def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
  """Returns 1 if the perceptron 'fires', 0 if not"""
  calculation = dot(weights, x) + bias
  return step_function(calculation)

importing Jupyter notebook from Chapter_04.ipynb


In [3]:
# AND Gate
and_weights = [2., 2]
and_bias = -3.

assert perceptron_output(and_weights, and_bias, [1,1]) == 1
assert perceptron_output(and_weights, and_bias, [0,1]) == 0
assert perceptron_output(and_weights, and_bias, [1,0]) == 0
assert perceptron_output(and_weights, and_bias, [0,0]) == 0

In [4]:
# OR Gate
or_weights = [2., 2]
or_bias = -1.

assert perceptron_output(or_weights, or_bias, [1,1]) == 1
assert perceptron_output(or_weights, or_bias, [1,0]) == 1
assert perceptron_output(or_weights, or_bias, [0,1]) == 1
assert perceptron_output(or_weights, or_bias, [0,0]) == 0

In [5]:
# NOT Gate
not_weights = [-2.]
not_bias = 1.

assert perceptron_output(not_weights, not_bias, [0]) == 1
assert perceptron_output(not_weights, not_bias, [1]) == 0

In [6]:
# XOR gate
and_gate = min
or_gate = max
xor_gate = lambda x, y: 0 if x== y else 1

In [7]:
import math

def sigmoid(t: float) -> float:
  return 1 / (1 + math.exp(-t))

In [8]:
def neuron_output(weights: Vector, inputs: Vector) -> float:
  # weights includes the bias term, inputs includes a 1
  return sigmoid(dot(weights, inputs))
  

In [9]:
from typing import List

def feed_forward(neural_network: List[List[Vector]],
                 input_vector: Vector) -> List[Vector]:
    """
    Feeds the input vector through the neural network.
    Returns the outputs of all layers (not just hte last one).
    """
    outputs: List[Vector] = []

    for layer in neural_network:
      input_with_bias = input_vector + [1] # Add a constant
      output = [neuron_output(neuron, input_with_bias) #Compute the output
                for neuron in layer]
      outputs.append(output)

      # Then the input to the next layer is the output of this one
      input_vector = output

    return outputs

    

In [10]:
xor_network = [# hidden layer
               [[20., 20, -30],   # 'and' neuron
                [20., 20, -10]],  # 'or' neuron
               # output layer
               [[-60., 60, -30]]] # '2nd input but no 1st input' neuron

# feed_forward returns the outputs of all layers, so the [-1] gets the
# final output, and the [0] gets the value out of the resulting vector
assert 0.000 < feed_forward(xor_network, [0,0])[-1][0] < 0.001
assert 0.999 < feed_forward(xor_network, [1,0])[-1][0] < 1.000
assert 0.999 < feed_forward(xor_network, [0,1])[-1][0] < 1.000
assert 0.000 < feed_forward(xor_network, [1,1])[-1][0] < 0.001